In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DFF1D3E320>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DFF1D642E0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello. How are you?")
]
result = model.invoke(messages)

In [4]:
result

AIMessage(content='Bonjour. Comment allez-vous ? \n\n\nYou can also use the more informal:\n\n**Salut. Comment vas-tu ?** \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 22, 'total_tokens': 53, 'completion_time': 0.056363636, 'prompt_time': 0.00014968, 'queue_time': 0.01392427, 'total_time': 0.056513316}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d2aee002-24d0-4cb7-b6e1-92f8e381fee6-0', usage_metadata={'input_tokens': 22, 'output_tokens': 31, 'total_tokens': 53})

In [5]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Bonjour. Comment allez-vous ? \n\n\nYou can also use the more informal:\n\n**Salut. Comment vas-tu ?** \n'

In [7]:
# Using LCEL - chain the components
chain = model | parser
chain.invoke(messages)

'Bonjour. Comment allez-vous ? \n'

In [8]:
# Prompt templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}"
prompt = ChatPromptTemplate.from_messages(
    [('system', generic_template),
     ('user', '{text}')]
)


In [9]:
result = prompt.invoke({'language': 'French', 'text': 'Hello'})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [10]:
result.to_messages()

[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [11]:
# Chaining together components with LCEL
chain = prompt | model | parser
chain.invoke({'language' : 'French', 'text': 'Hello'})

'Bonjour \n'